# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet

# 1. Init Dataset

In [3]:
#sample dataset
import numpy as np

image_shape = [512, 512]
n_class = 21 #background + 20 label
total_size = 100

tr_x = ["a.jpg" for _ in range(total_size)]
tr_y = ["a.xml" for _ in range(total_size)]

# 2. Load Dataset & Apply Transform

In [4]:
def load(x_true, y_true, mask = True):
    max_pad_size = np.random.randint(10) + 1
    
    image = np.random.random([*image_shape, 3]).astype(np.float32)
    y_true = np.random.randint(n_class, size = [max_pad_size, 1]).astype(np.int32)
    bbox_true = np.array([tfdet.core.bbox.random_bbox(image_shape = image_shape) for _ in range(max_pad_size)], dtype = np.int32)
    result = [image, y_true, bbox_true]
    if mask:
        mask_true = np.random.randint(2, size = (max_pad_size, *image_shape, 1)).astype(np.uint8)
        result = [image, y_true, bbox_true, mask_true]
    return tuple(result)
        
tr_dataset = tfdet.dataset.Dataset(tr_x, tr_y, transform = [{"name":load, "mask":True}, #or load,
                                                            {"name":"resize", "image_shape":image_shape},
                                                            {"name":"pad", "image_shape":image_shape},
                                                            {"name":"filter_annotation"},
                                                            #{"name":"label_encode", "label":None},
                                                            {"name":"normalize", "mean":[123.675, 116.28, 103.53], "std":[58.395, 57.12, 57.375]}])
out = tr_dataset[0] #or next(iter(tr_dataset))
x_true, y_true, bbox_true = out[:3]
mask_true = out[3] if 3 < len(out) else None

if mask_true is None:
    print("x_true:{0}, y_true:{1}, bbox_true:{2}".format(x_true.shape, y_true.shape, bbox_true.shape))
else:
    print("x_true:{0}, y_true:{1}, bbox_true:{2}, mask_true:{3}".format(x_true.shape, y_true.shape, bbox_true.shape, mask_true.shape))

x_true:(512, 512, 3), y_true:(2, 1), bbox_true:(2, 4), mask_true:(2, 512, 512, 1)


# 3. Build Pipeline

In [5]:
tr_pipe = tfdet.dataset.PipeLoader(tr_dataset)
#tr_pipe = tfdet.dataset.pipeline.args2dict(tr_pipe) #for train_model
#tr_pipe = tfdet.dataset.pipeline.collect(tr_pipe) #filtered item by key
tr_pipe = tfdet.dataset.pipeline.key_map(tr_pipe, batch_size = 4, prefetch = True)

out = next(iter(tr_pipe))
image, y_true, bbox_true = out[:3]
mask_true = out[3] if 3 < len(out) else None

if mask_true is None:
    print("x_true:{0}, y_true:{1}, bbox_true:{2}".format(image.shape, y_true.shape, bbox_true.shape))
else:
    print("x_true:{0}, y_true:{1}, bbox_true:{2}, mask_true:{3}".format(image.shape, y_true.shape, bbox_true.shape, mask_true.shape))

x_true:(4, 512, 512, 3), y_true:(4, 10, 1), bbox_true:(4, 10, 4), mask_true:(4, 10, 512, 512, 1)
